In [1]:
from pathlib import Path

import pyautogui
import cv2
from PIL import Image

from paddleocr import PaddleOCR


c:\Honey\Projects\semi\yuzyproject-aimodels\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [9]:
SCREENSHOT_PATH = Path("screenshot_full.png")

def capture_fullscreen(save_path: Path = SCREENSHOT_PATH) -> Path:
    """
    전체 화면을 캡처해 PNG로 저장하고 경로를 반환합니다.
    """
    screenshot = pyautogui.screenshot()
    screenshot.save(save_path)
    return save_path

# 한 번 캡쳐 실행
img_path = capture_fullscreen()
print(f"캡처 저장 위치: {img_path.resolve()}")


캡처 저장 위치: C:\Honey\Projects\semi\yuzyproject-aimodels\screenshot_full.png


In [10]:
CROP_PATH = Path("screenshot_crop.png")

def select_and_crop(full_image_path: Path, save_path: Path = CROP_PATH) -> Path:
    """
    전체 캡쳐 이미지에서 드래그로 영역(ROI)을 선택하고, 
    그 부분만 잘라서 저장합니다.
    """
    img = cv2.imread(str(full_image_path))
    if img is None:
        raise FileNotFoundError(f"이미지를 찾을 수 없습니다: {full_image_path}")

    # ROI 선택: 드래그 → Enter/Space로 확정, ESC로 창 닫기
    r = cv2.selectROI(
        "Drag region then press ENTER (ESC to cancel)", 
        img, 
        showCrosshair=True, 
        fromCenter=False
    )
    x, y, w, h = r

    cropped = img[y : y + h, x : x + w]

    cv2.destroyAllWindows()

    cv2.imwrite(str(save_path), cropped)
    return save_path

crop_path = select_and_crop(img_path)
print(f"크롭 이미지 저장 위치: {crop_path.resolve()}")


크롭 이미지 저장 위치: C:\Honey\Projects\semi\yuzyproject-aimodels\screenshot_crop.png


In [7]:
ocr = PaddleOCR(
    lang='korean'
)

Creating model: ('PP-LCNet_x1_0_doc_ori', None)
Model files already exist. Using cached files. To redownload, please delete the directory manually: `C:\Users\hubo0\.paddlex\official_models\PP-LCNet_x1_0_doc_ori`.
Creating model: ('UVDoc', None)
Model files already exist. Using cached files. To redownload, please delete the directory manually: `C:\Users\hubo0\.paddlex\official_models\UVDoc`.
Creating model: ('PP-LCNet_x1_0_textline_ori', None)
Model files already exist. Using cached files. To redownload, please delete the directory manually: `C:\Users\hubo0\.paddlex\official_models\PP-LCNet_x1_0_textline_ori`.
Creating model: ('PP-OCRv5_server_det', None)
Model files already exist. Using cached files. To redownload, please delete the directory manually: `C:\Users\hubo0\.paddlex\official_models\PP-OCRv5_server_det`.
Creating model: ('korean_PP-OCRv5_mobile_rec', None)
Model files already exist. Using cached files. To redownload, please delete the directory manually: `C:\Users\hubo0\.padd

In [8]:
def ocr_text_only(image_path: Path) -> str:
    """
    PaddleOCR로 이미지에서 텍스트만 뽑아서 한 문자열로 합칩니다.
    """
    result = ocr.ocr(str(image_path), cls=True)

    lines = []
    # result는 [ [ [box, (text, score)], ... ] ] 이런 구조입니다.
    for block in result:
        for line in block:
            text = line[1][0]   # 인식된 텍스트
            # score = line[1][1]  # 신뢰도(0~1), 필요하면 쓰세요
            lines.append(text)

    return "\n".join(lines)

extracted_text = ocr_text_only(crop_path)
print("=== OCR 결과 (텍스트만) ===")
print(extracted_text)


C:\Users\hubo0\AppData\Local\Temp\ipykernel_8512\3461472558.py:5: DeprecationWarning: Please use `predict` instead.
  result = ocr.ocr(str(image_path), cls=True)


TypeError: PaddleOCR.predict() got an unexpected keyword argument 'cls'

In [ ]:
import paddleocr, paddle
print(paddleocr.__version__)
print(paddle.__version__)

3.3.2
3.2.0
